In [1]:
%load_ext autoreload
%autoreload 2

### Setup

In [2]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from utils import DATA_DIR, ROOT_DIR, PLOT_DIR
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from dataloader import get_nd_array, get_slice
from download.hf import pull_predictions_from_hf

import metaanalysis
plt.close()

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)  # ignore fitting warnings

# Display all columns
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', None)

### Load Data

In [3]:
local_path = f'{DATA_DIR}/benchmarks.parquet'
df_benchmarks = pd.read_parquet(local_path)
print(f'Loaded {len(df_benchmarks):,} model evaluations')

Loaded 810,570 model evaluations


In [4]:
from utils import get_selected_tasks
benchmark_tasks = set(df_benchmarks['task'].unique())
TASKS = sorted(list(benchmark_tasks))


In [5]:
from datadecide import get_compute
from utils import extract_flops
from utils.constants_models import DDOS_MODEL_NAMES

df_benchmarks["model_path"] = df_benchmarks["model_config"].apply(lambda x: x["model_path"])
df_benchmarks[["flops", "observational_model"]] = df_benchmarks["model_path"].apply(extract_flops).apply(pd.Series)

observational_models = sorted(df_benchmarks[df_benchmarks['observational_model'] == True]['model'].unique())
datadecide_models = DDOS_MODEL_NAMES

# Add FLOPs col for DataDecide
datadecide_mask = df_benchmarks['model'].isin(datadecide_models)
df_benchmarks.loc[datadecide_mask, 'flops'] = df_benchmarks.loc[datadecide_mask, 'size'].apply(get_compute).astype('float64')

In [6]:
# Update model column to model_path for entries with -model-merged
    # last-29-model-merged => peteish7-last-29-model-merged
mask = df_benchmarks['model_path'].str.contains('-model-merged', na=False)
df_benchmarks.loc[mask, 'model'] = df_benchmarks.loc[mask, 'model_path'].str.split('/').str[-2:].str.join('-')

### Investigate Benchmarks

In [7]:
mmlu      = [t for t in TASKS if 'mmlu' in t and ':' not in t and '_pro_' not in t]
minerva   = [t for t in TASKS if 'minerva' in t and ':' not in t and 'math_500' not in t and t != 'minerva']
mmlu_pro  = [t for t in TASKS if '_pro_' in t and ':rc' in t]
mmlu_mc   = [t for t in TASKS if 'mmlu' in t and ':mc' in t and '_pro_' not in t]
olmes     = ['arc_challenge', 'arc_easy', 'boolq', 'csqa', 'hellaswag', 'openbookqa', 'piqa', 'socialiqa', 'winogrande']
olmes_mc  = [f'{task}:mc' for task in olmes]
olmes_para        = [f'{task}:para' for task in olmes]
olmes_distractors = [f'{task}:distractors' for task in olmes]
olmes_enlarge     = [f'{task}:enlarge' for task in olmes]
olmes_gen = ['drop', 'gsm8k', 'jeopardy', 'squad', 'triviaqa'] # naturalqs
agi_eval  = [t for t in TASKS if 'agi_eval' in t and ':rc' in t]
bbh       = [t for t in TASKS if 'bbh' in t and ':' not in t]
paloma    = [t for t in TASKS if 'paloma' in t]
llm_compression = [t for t in TASKS if 'llm_compression' in t]
custom_loss = [t for t in TASKS if 'custom_loss' in t]

In [8]:
from utils import get_title_from_task

def add_multitask_avg(df_benchmarks, task_set):
    _slice = get_slice(df_benchmarks, task=task_set)

    new_task = _slice.fillna('')
    new_task = new_task.groupby(['model', 'step', 'mix', 'size'])[['model_config', 'primary_score', 'logits_per_byte_corr', 'logits_per_char_corr']]
    new_task = new_task.agg(lambda x: x.iloc[0] if x.name == 'model_config' else x[pd.to_numeric(x, errors='coerce').notnull()].mean())
    new_task = new_task.reset_index()

    new_task['step'] = pd.to_numeric(new_task['step'], errors='coerce') 
    # new_task['size'] = pd.to_numeric(new_task['size'], errors='coerce')

    new_task['task'] = get_title_from_task(task_set)

    # Append the aggregate task entries to df_benchmarks
    df_benchmarks = pd.concat([df_benchmarks, new_task], axis=0)

    return df_benchmarks

# new task suites
multitask_math = ["gsm_plus", "gsm_symbolic_main", "gsm_symbolic_p1", "gsm_symbolic_p2", "minerva_math_500"] + ['gsm8k'] + ['minerva'] # 6 # "aime"
multitask_code = ['mbpp', 'mbppplus', 'codex_humaneval', 'codex_humanevalplus'] # 4
multitask_knowledge = ["medmcqa", 'autobencher'] + olmes + ['mmlu'] + olmes_gen + ['mmlu_pro'] + ['agi_eval'] # 19
multitask = multitask_knowledge + multitask_math + multitask_code # 30 # bbh
olmes_all = olmes + mmlu + olmes_gen

# Re-order tasks so that the title logic works (a bit hacky, yes)
olmes_all = ['jeopardy'] + list(set(olmes_all) - {'jeopardy'})
multitask = ['boolq'] + list(set(multitask) - {'boolq'})

df_benchmarks = add_multitask_avg(df_benchmarks, task_set=mmlu)
df_benchmarks = add_multitask_avg(df_benchmarks, task_set=mmlu_pro)
df_benchmarks = add_multitask_avg(df_benchmarks, task_set=agi_eval)
df_benchmarks = add_multitask_avg(df_benchmarks, task_set=minerva)
df_benchmarks = add_multitask_avg(df_benchmarks, task_set=multitask_math)
df_benchmarks = add_multitask_avg(df_benchmarks, task_set=multitask_code)
df_benchmarks = add_multitask_avg(df_benchmarks, task_set=multitask_knowledge)
df_benchmarks = add_multitask_avg(df_benchmarks, task_set=multitask)

/var/folders/cr/z51hvhb932vbx29jw5hxd9nh0000gn/T/ipykernel_58552/394494844.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_task = _slice.fillna('')
/var/folders/cr/z51hvhb932vbx29jw5hxd9nh0000gn/T/ipykernel_58552/394494844.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_task = _slice.fillna('')
/var/folders/cr/z51hvhb932vbx29jw5hxd9nh0000gn/T/ipykernel_58552/394494844.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) inst

In [9]:
selected_tasks = get_selected_tasks(TASKS) + ['multitask_all', 'multitask_math', 'multitask_code', 'multitask_knowledge']

In [10]:
from metaanalysis import compute_metaproperties

df_results = compute_metaproperties(
    # df_benchmarks, None, [multitask_math, multitask_code, multitask_knowledge, multitask, olmes_all],
    # df_benchmarks, None, [olmes], 
    df_benchmarks, None, selected_tasks, 
    run_irt=False, run_instance_analysis=False, 
    use_parallel=True, quiet=True
)

Computing benchmark properties:   8%|▊         | 3/37 [07:09<54:50, 96.79s/it]   

bpb is empty array: [] on model "peteish-moreeval-1B-1xC" for task "aime"
bpb is empty array: [] on model "peteish-moreeval-1B-1xC" for task "aime"
aime failed on ladder fits No scores found for model=peteish-moreeval-1B-1xC, metric=['logits_per_byte_corr', 'primary_score'], task=aime. Seeing: []
Failed to calculate compute cost: num_instances should be constant across task=multitask_all for task_as_list=['multitask_all']
Failed to calculate compute cost: num_instances should be constant across task=multitask_math for task_as_list=['multitask_math']
Failed to calculate compute cost: num_instances should be constant across task=multitask_code for task_as_list=['multitask_code']
Failed to calculate compute cost: num_instances should be constant across task=multitask_knowledge for task_as_list=['multitask_knowledge']


Computing benchmark properties: 100%|██████████| 37/37 [13:34<00:00, 22.01s/it]  


In [11]:
from utils import get_pretty_task_name

# sizes = ['1B', '1B-100B', '13B', '32B']
sizes = ['1B-100B']
metric_a = 'primary_score'
metric_b = 'logits_per_byte_corr'

# '60M', '150M'
# In order to add DataDecide, we need step_rel_std:last30 numbers

ERROR_METRICS_PRIMARY = [
    f'dec_acc:{metric_a}:150M',
    f'rel_error:step_2:13B:bpb_to_primary', # metric_b
]

ERROR_METRICS_BPB = [
    f'dec_acc:{metric_b}:150M',
    f'rel_error:step_1:13B:bpb_to_primary',
]

table_tasks = df_results.index

# Collect all data into a dict
data = {}
for task in table_tasks:
    task_data = df_results.loc[task]
    data[task] = {}
    # Add noise measures for each size
    for size in sizes:
        data[task][size] = {}
        for col in [
            f'snr:{metric_a}:{size}', 
            f'rel_std:{metric_a}:{size}', 
            f'step_rel_std:last30:{metric_a}:{size}',
            f'snr:{metric_b}:{size}', 
            f'rel_std:{metric_b}:{size}', 
            f'step_rel_std:last30:{metric_b}:{size}',
        ]:
            if col in task_data.index:
                col_no_size = col.replace(f':{size}', '')
                data[task][size][col_no_size] = task_data[col]
        
    for col in ERROR_METRICS_PRIMARY + ERROR_METRICS_BPB:
        if col in task_data.index:
            if 'error' not in data[task]:
                data[task]['error'] = {}
            data[task]['error'][col] = task_data[col]
        

def pretty_metric_name(metric):
    PRETTY_METRIC_NAME = {
        'primary_score': 'Primary',
        'logits_per_byte_corr': 'BPB',
        'dec_acc:primary_score:150M': 'Primary Dec Acc 150M (↑)',
        'dec_acc:logits_per_byte_corr:150M': 'BPB Dec Acc 150M (↑)',
        'rel_error:step_1:13B:bpb_to_primary': 'BPB % Err 13B (↓)',
        'rel_error:step_2:13B:bpb_to_primary': 'Primary % Err 13B (↓)',
    }
    return PRETTY_METRIC_NAME.get(metric, metric)

# Convert to readable metrics
rows = []
for task, task_data in data.items():
    row = {'task': get_pretty_task_name(task) } # pretty_metric_name(metric)
    for metric, err_metrics in zip([metric_a, metric_b], [ERROR_METRICS_PRIMARY, ERROR_METRICS_BPB]):
        for size in sizes:
            snr          = task_data[size].get(f'snr:{metric}', '--')
            rel_std      = task_data[size].get(f'rel_std:{metric}', '--') 
            step_rel_std = task_data[size].get(f'step_rel_std:last30:{metric}', '--')
            
            if all(x != '--' for x in [snr, rel_std, step_rel_std]):
                row[size + ' ' + pretty_metric_name(metric)] = f'${snr:.1f}$'
                # row[size + ' ' + pretty_metric_name(metric)] = f'${snr:.1f}_{{{rel_std:.3f} / {step_rel_std:.3f}}}$'
            else:
                row[size + ' ' + pretty_metric_name(metric)] = '--'
    
        for err_metric in err_metrics:
            error = task_data['error'][err_metric]

            if 'dec_acc' in err_metric:
                err_str = f'${error*100:.1f}$'
            # elif 'rel_error' in err_metric:
            #     margin_of_err_metric = {
            #         'rel_error:step_2:13B:bpb_to_primary': 'scaling_margin_of_error:stacked:13B:bpb_to_primary',
            #         'rel_error:step_1:13B:bpb_to_primary': 'scaling_margin_of_error:step_1:13B:bpb_to_primary'
            #     }
            #     moe = df_results.loc[task, margin_of_err_metric[err_metric]]
            #     err_str = f'${error*100:.1f}%_{{\pm {moe*100:.1f}%}}$'
            else:
                err_str = f'${error*100:.1f}$'
            
            row[pretty_metric_name(err_metric)] = err_str
        
    rows.append(row)

df_noise_bpb = pd.DataFrame(rows)
df_noise_bpb = df_noise_bpb.set_index('task')

# Sort cols
df_noise_bpb = df_noise_bpb.reindex([
    "1B-100B Primary",
    "1B-100B BPB",
    "Primary % Err 13B (↓)",
    "BPB % Err 13B (↓)",
    "Primary Dec Acc 150M (↑)",
    "BPB Dec Acc 150M (↑)",
], axis=1)

# Remove certain tasks
df_noise_bpb = df_noise_bpb[~(
    df_noise_bpb.index.str.contains(':mc', case=False) |
    df_noise_bpb.index.str.contains('paloma', case=False) |
    df_noise_bpb.index.str.contains('aime', case=False)
)]

# Sort by Primary column, converting string values like '$8.16$' to floats
df_noise_bpb = df_noise_bpb.sort_values(
    by='1B-100B Primary', 
    key=lambda x: x.str.extract(r'\$([\d.]+)\$')[0].astype(float), 
    ascending=False
)

# Bold higher values between Primary and BPB columns
col_pairs = [
    ['1B-100B Primary', '1B-100B BPB'],
    ['Primary Dec Acc 150M (↑)', 'BPB Dec Acc 150M (↑)'],
    ['Primary % Err 13B (↓)', 'BPB % Err 13B (↓)']
]
for col_a, col_b in col_pairs:
    for idx in df_noise_bpb.index:
        col_a_val = df_noise_bpb.loc[idx, col_a]
        col_b_val = df_noise_bpb.loc[idx, col_b]

        def get_val(_str):
            _str = _str.strip('$')
            _str = _str.split('_')[0]
            _str = _str.strip('%')
            return _str

        val_a = float(get_val(col_a_val))
        val_b = float(get_val(col_b_val))

        if abs(val_a - val_b) < 0.01:  # Values are effectively equal
            df_noise_bpb.loc[idx, col_a] = f'{col_a_val}'
            df_noise_bpb.loc[idx, col_b] = f'{col_b_val}'
        elif 'Err' in col_a:  # Lower is better for error metrics
            if val_a < val_b:
                df_noise_bpb.loc[idx, col_a] = f'$\\mathbf{{{col_a_val.strip("$")}}}$'
                df_noise_bpb.loc[idx, col_b] = f'{col_b_val}'
            else:
                df_noise_bpb.loc[idx, col_a] = f'{col_a_val}'
                df_noise_bpb.loc[idx, col_b] = f'$\\mathbf{{{col_b_val.strip("$")}}}$'
        else:  # Higher is better for non-error metrics
            if val_a > val_b:
                df_noise_bpb.loc[idx, col_a] = f'$\\mathbf{{{col_a_val.strip("$")}}}$'
                df_noise_bpb.loc[idx, col_b] = f'{col_b_val}'
            else:
                df_noise_bpb.loc[idx, col_a] = f'{col_a_val}'
                df_noise_bpb.loc[idx, col_b] = f'$\\mathbf{{{col_b_val.strip("$")}}}$'

df_noise_bpb

olmes_core9
minerva
olmes_gen
mmlu
mmlu_pro
agi_eval
bbh
arc_challenge
arc_easy
boolq
csqa
hellaswag
openbookqa
piqa
socialiqa
winogrande
drop
gsm8k
jeopardy
squad
triviaqa
mbpp
mbppplus
codex_humaneval
codex_humanevalplus
autobencher
gsm_plus
gsm_symbolic_main
gsm_symbolic_p1
gsm_symbolic_p2
medmcqa
minerva_math_500
aime
multitask_all
multitask_math
multitask_code
multitask_knowledge


1B-100B Primary      1B-100B BPB Primary % Err 13B (↓)  \
task                                                                       
TriviaQA                   $27.9$  $\mathbf{61.8}$                 $2.5$   
SQuAD                      $23.8$  $\mathbf{29.0}$        $\mathbf{7.6}$   
OLMES Gen         $\mathbf{23.1}$           $20.6$        $\mathbf{0.9}$   
ARC Easy                   $21.0$  $\mathbf{64.6}$                 $5.3$   
Jeopardy                   $20.2$  $\mathbf{22.6}$        $\mathbf{3.5}$   
AutoBencher                $15.9$  $\mathbf{31.3}$        $\mathbf{0.2}$   
Knowledge Tasks            $13.7$  $\mathbf{44.3}$        $\mathbf{0.8}$   
HellaSwag                  $11.8$  $\mathbf{14.9}$                 $1.4$   
DROP              $\mathbf{11.5}$            $9.9$                $59.0$   
MMLU Pro                   $11.0$  $\mathbf{27.6}$                 $2.7$   
All Tasks                  $10.0$  $\mathbf{31.5}$                 $2.3$   
MMLU                        $9.8$  $\mathbf{35.9}$                 $4.3$   
ARC Challenge               $6.6$  $\mathbf{44.8}$                 $9.7$   
HumanEval                   $6.1$  $\mathbf{25.1}$                 $9.2$   
Code Tasks                  $5.5$  $\mathbf{42.0}$                $29.5$   
CommonsenseQA               $5.5$  $\mathbf{41.9}$        $\mathbf{3.6}$   
SocialIQA                   $5.5$  $\mathbf{48.0}$        $\mathbf{0.4}$   
HumanEval+                  $5.5$  $\mathbf{27.4}$                $29.7$   
OLMES Core 9                $5.4$  $\mathbf{73.2}$                 $3.7$   
WinoGrande         $\mathbf{4.6}$            $3.6$                $10.3$   
PIQA                        $4.2$   $\mathbf{8.8}$        $\mathbf{0.5}$   
BBH                $\mathbf{3.6}$            $2.5$                $67.1$   
MedMCQA                     $3.5$  $\mathbf{29.5}$                 $8.8$   
AGI Eval                    $2.5$  $\mathbf{19.5}$                $13.7$   
OpenBookQA                  $2.1$  $\mathbf{24.2}$                 $7.7$   
MBPP                        $2.0$  $\mathbf{41.8}$                $23.6$   
Minerva MATH                $1.9$  $\mathbf{88.6}$                $11.9$   
GSM+                        $1.8$   $\mathbf{7.3}$                $20.0$   
Math Tasks                  $1.8$  $\mathbf{22.6}$                $46.0$   
MBPP+                       $1.7$  $\mathbf{30.8}$                $39.5$   
GSM Symbolic P1             $1.6$   $\mathbf{6.6}$               $538.6$   
BoolQ                       $1.5$  $\mathbf{64.8}$        $\mathbf{5.1}$   
Minerva MATH 500            $1.4$  $\mathbf{90.5}$                $52.5$   
GSM Symbolic                $1.3$   $\mathbf{6.5}$                $83.0$   
GSM8K                       $1.2$   $\mathbf{7.0}$                $38.6$   
GSM Symbolic P2             $1.0$   $\mathbf{7.0}$                $74.8$   

                 BPB % Err 13B (↓) Primary Dec Acc 150M (↑)  \
task                                                          
TriviaQA            $\mathbf{0.5}$                   $68.3$   
SQuAD                       $27.8$                   $59.7$   
OLMES Gen                    $2.6$                   $63.3$   
ARC Easy            $\mathbf{0.8}$                   $93.0$   
Jeopardy                    $18.6$                   $82.0$   
AutoBencher                  $4.5$                   $89.3$   
Knowledge Tasks              $1.0$                   $79.0$   
HellaSwag           $\mathbf{1.0}$                   $74.3$   
DROP               $\mathbf{11.3}$                   $57.3$   
MMLU Pro            $\mathbf{1.3}$                   $83.0$   
All Tasks           $\mathbf{0.4}$                   $77.0$   
MMLU                $\mathbf{0.4}$                   $89.0$   
ARC Challenge       $\mathbf{2.1}$                   $83.3$   
HumanEval           $\mathbf{7.9}$                   $74.3$   
Code Tasks          $\mathbf{9.7}$                   $80.3$   
CommonsenseQA                $5.9$          $\mathbf{68.7}

In [13]:
CAPTION = "SNR using BPB"
table_str = df_noise_bpb.rename_axis('Task').fillna('--').to_latex(
    label="tab:noise",
    escape=False,
    float_format=lambda x: '{:.3f}'.format(x),
    column_format='p{0.18\\textwidth}' + 'C{0.05\\textwidth}'*len(df_noise_bpb.columns)
)
table_str = table_str.replace('%', '\%')
table_str = table_str.replace('table', 'table*')\
    .replace('\\begin{table*}', '\\begin{table*}\n\\scriptsize\n\\centering\n\\makebox[\\textwidth]{')\
    .replace('\\end{table*}', '}\n\\caption{' + CAPTION + '}\n\\end{table*}')\
    .replace('Symbolic', 'Symb.')

print(table_str)

\begin{table*}
\scriptsize
\centering
\makebox[\textwidth]{
\label{tab:noise}
\begin{tabular}{p{0.18\textwidth}C{0.05\textwidth}C{0.05\textwidth}C{0.05\textwidth}C{0.05\textwidth}C{0.05\textwidth}C{0.05\textwidth}}
\toprule
 & 1B-100B Primary & 1B-100B BPB & Primary \% Err 13B (↓) & BPB \% Err 13B (↓) & Primary Dec Acc 150M (↑) & BPB Dec Acc 150M (↑) \\
Task &  &  &  &  &  &  \\
\midrule
TriviaQA & $27.9$ & $\mathbf{61.8}$ & $2.5$ & $\mathbf{0.5}$ & $68.3$ & $\mathbf{85.3}$ \\
SQuAD & $23.8$ & $\mathbf{29.0}$ & $\mathbf{7.6}$ & $27.8$ & $59.7$ & $\mathbf{61.7}$ \\
OLMES Gen & $\mathbf{23.1}$ & $20.6$ & $\mathbf{0.9}$ & $2.6$ & $63.3$ & $\mathbf{67.3}$ \\
ARC Easy & $21.0$ & $\mathbf{64.6}$ & $5.3$ & $\mathbf{0.8}$ & $93.0$ & $93.0$ \\
Jeopardy & $20.2$ & $\mathbf{22.6}$ & $\mathbf{3.5}$ & $18.6$ & $82.0$ & $\mathbf{83.0}$ \\
AutoBencher & $15.9$ & $\mathbf{31.3}$ & $\mathbf{0.2}$ & $4.5$ & $89.3$ & $89.3$ \\
Knowledge Tasks & $13.7$ & $\mathbf{44.3}$ & $\mathbf{0.8}$ & $1.0$ & $79.0$ &